# Import

In [1]:
import os
import matplotlib.pyplot as plt
plt.close('all')
import dfm_tools as dfmt
import hydrolib.core.dflowfm as hcdfm
import xarray as xr
import pandas as pd
import numpy as np
import contextily as ctx

import solara
from ipyleaflet import Map, basemaps
import datetime


# User Input

In [ ]:
####################################### User Input ########################################
model_name_options = ["Vietnam", "Bonaire"]
model_name = solara.reactive("Vietnam")
m = solara.reactive(Map(center=(18.125, 106.325), zoom=10, scroll_wheel_zoom=True, 
                        basemap=basemaps.OpenStreetMap.Mapnik))

lat_min = solara.reactive(0.0); lat_max = solara.reactive(0.0); lon_min = solara.reactive(0.0); lon_max = solara.reactive(0.0)
def reset_map():
    if model_name.value == "Bonaire":
        lon_min.set(-68.55); lon_max.set(-67.9); lat_min.set(11.8); lat_max.set(12.6)
    elif model_name.value == "Vietnam":
        lon_min.set(105.8); lon_max.set(106.85); lat_min.set(17.75); lat_max.set(18.5)
    m.set(Map(center=(lat_min.value+(lat_max.value-lat_min.value)*0.5, lon_min.value+(lon_max.value-lon_min.value)*0.5), 
              zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik))

# ------------------------------ copied from modelbuilder_example.ipynb ------------------------------
dxy = 0.5
dir_output = os.path.abspath(f'./{model_name}_model')
overwrite = False # used for downloading of forcing data. Always set to True when changing the domain
crs = 'EPSG:4326' # coordinate reference system
# make directories and list all files
os.makedirs(dir_output, exist_ok=True)
dir_output_data = os.path.join(dir_output, 'data')
os.makedirs(dir_output_data, exist_ok=True)
# ----------------------------------------------------------------------------------------------------

# to get dates into desired format of '2022-11-01', do: date_min.value.strftime('%Y-%m-%d')
date_min = solara.reactive(datetime.date(2022, 11, 1))
date_max = solara.reactive(datetime.date(2022, 11, 3))
ref_date = solara.reactive(datetime.date(2022, 1, 1))

@solara.component
def Tab_User_Input():
    solara.use_effect(reset_map, dependencies=[model_name.value])
    with solara.Card("User Input", style={"width": "100%", "padding": "10px"}):
        solara.Select(label="Model Name", value=model_name, values=model_name_options)

        solara.Text("Select min date:"); solara.lab.InputDate(date_min)
        solara.Text("Select max date:"); solara.lab.InputDate(date_max)
        solara.Text("Select reference date:"); solara.lab.InputDate(ref_date)
        if date_max.value < date_min.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", 
                            style={"color": "red"})

# Grid Generation and Refinement

In [ ]:
#continue here

# generate spherical regular grid
mk_object = dfmt.make_basegrid(lon_min.value, lon_max.value, lat_min.value, lat_max.value, dx=dxy, dy=dxy, crs=crs)


# generate plifile from grid extent and coastlines
bnd_gdf = dfmt.generate_bndpli_cutland(mk=mk_object, res='h', buffer=0.01)
bnd_gdf_interp = dfmt.interpolate_bndpli(bnd_gdf, res=0.03)
pli_polyfile = dfmt.geodataframe_to_PolyFile(bnd_gdf_interp, name=f'{model_name.value}_bnd')
poly_file = os.path.join(dir_output, f'{model_name}.pli')
pli_polyfile.save(poly_file)

# Putting everything together

put code of single tabs in own code cell, per tab --> to have it more structured!

In [ ]:



###########################################################################################

@solara.component
def Tab2():
    
    with solara.Card("xxx2", style={"width": "100%", "padding": "10px"}):
        solara.Text("text")

@solara.component
def Tab3():
    
    with solara.Card("xxx3", style={"width": "100%", "padding": "10px"}):
        solara.Text("text")


@solara.component
def Tabxxx():
    
    with solara.Card("xxx", style={"width": "100%", "padding": "10px"}):
        solara.Text("text")

###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
selected_tab = solara.reactive('UserInput') 
@solara.component
def SettingsTabs():
    with solara.Column(style={"width": "100%", "align-items": "center"}):
        with solara.Row(gap="10px", style={"justify-content": "flex-start", "width": "80%"}):
            solara.Button("User Input", on_click=lambda: selected_tab.set('UserInput'))
            solara.Button("2", on_click=lambda: selected_tab.set('2'))
            solara.Button("3", on_click=lambda: selected_tab.set('3'))
        with solara.Row(gap="10px", style={"justify-content": "flex-start", "width": "80%"}):
            solara.Button("xxx", on_click=lambda: selected_tab.set('xxx'))
 
    if selected_tab.value == 'UserInput':
        Tab_User_Input()
    elif selected_tab.value == '2':
        Tab2()
    elif selected_tab.value == '3':
        Tab3()
    elif selected_tab.value == 'xxx':
        Tabxxx()

@solara.component
def Page():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "650px"}):
            display(m.value)

        with solara.Column(style={"width": "30%", "min-width": "500px"}):
            SettingsTabs()

Page()